<a href="https://colab.research.google.com/github/lamkaiyi/RAG_with_Llama/blob/main/HuggingFace_LangChain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch\
torchvision\
transformers\
langchain\
ragas\
arxiv\
pymupdf\
chromadb\
wandb\
tiktoken\
sentence-transformers\
bitsandbytes\
accelerate\
ragas\
llama_index\
datasets\
text-generation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.0 MB/s 

##Import Relevant Dependencies

In [ ]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device) #check that cuda is detected

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 16}
)


cuda:0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Load in relevant documents

In [ ]:
from langchain.document_loaders import ArxivLoader

topic = "Support Vector Machines"
n = 7  #retrieve top n arXiV papers on the topic

base_docs = ArxivLoader(query=topic, load_max_docs=n).load()
len(base_docs)

7

In [ ]:
for doc in base_docs:
  print(doc.metadata)

{'Published': '2016-05-31', 'Title': 'Linear Classification of data with Support Vector Machines and Generalized Support Vector Machines', 'Authors': 'Xiaomin Qi, Sergei Silvestrov, Talat Nazir', 'Summary': 'In this paper, we study the support vector machine and introduced the notion\nof generalized support vector machine for classification of data. We show that\nthe problem of generalized support vector machine is equivalent to the problem\nof generalized variational inequality and establish various results for the\nexistence of solutions. Moreover, we provide various examples to support our\nresults.'}
{'Published': '2011-11-03', 'Title': 'Qualitative Robustness of Support Vector Machines', 'Authors': 'Robert Hable, Andreas Christmann', 'Summary': "Support vector machines have attracted much attention in theoretical and in\napplied statistics. Main topics of recent interest are consistency, learning\nrates and robustness. In this article, it is shown that support vector machines\nare

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200,
                                            separators=["\n\n", "\n", " ", ""]
                                               ) #to split docs into chunks

docs = text_splitter.split_documents(base_docs)

vectorstore = Chroma.from_documents(docs, embed_model) #store chunks in vector db

In [ ]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k" : 2}) #set to retrieve top 2 most relevant documents

In [ ]:
# ask a simple question about the topic
relevant_docs = base_retriever.get_relevant_documents(f"What is {topic} and where is it commonly applied?")


In [ ]:
len(relevant_docs)

2

In [ ]:
from google.colab import userdata
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = userdata.get('huggingfaceAPIkey')

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)


model.eval()
print(f"Model loaded on {device}")



/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1033: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded on cuda:0


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:

generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',

    temperature=0.01,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=1000,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)



In [ ]:
# test the text generation pipeline on a random question

res = generate_text("Explain to me the difference between nuclear fission and fusion.")
print(res[0]["generated_text"])



Explain to me the difference between nuclear fission and fusion.

Nuclear fission is a process in which an atomic nucleus splits into two or more smaller nuclei, releasing a large amount of energy in the process. This process typically occurs when an atom is bombarded with a high-energy particle, such as a neutron. When the nucleus splits, it releases a large amount of energy in the form of kinetic energy of the fragments and gamma radiation.

Nuclear fusion, on the other hand, is the process by which two or more atomic nuclei combine to form a single, heavier nucleus. This process also releases a large amount of energy, but it does so at much higher temperatures than those required for fission. In order to achieve fusion, the atoms must be heated to incredibly high temperatures, typically over 100 million degrees Celsius.

One key difference between fission and fusion is the direction of the energy release. In fission, the energy is released outward from the nucleus, while in fusion, 

In [ ]:
# place inside a LangChain pipeline
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)


In [ ]:
# test on a random question

llm(prompt="Explain to me the difference between a derivative and a futures contract.")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nA derivative is a financial instrument that derives its value from an underlying asset, such as a stock, bond, commodity, or currency. Derivatives can be used to hedge risk, speculate on price movements, or generate income. Common types of derivatives include options, futures, swaps, and forwards.\n\nA futures contract, on the other hand, is a type of derivative that obligates the buyer and seller to purchase and sell a specific asset at a predetermined price on a specific date in the future. Futures contracts are standardized, meaning they specify the quantity, quality, and delivery date of the underlying asset. They are traded on regulated exchanges and are subject to margin requirements and daily settlement.\n\nThe key differences between a derivative and a futures contract are:\n\n1. Obligation: A futures contract is a legally binding agreement, whereas a derivative is not necessarily obligatory.\n2. Standardization: Futures contracts are standardized, while derivatives can be


#Initializing a RetrievalQA Chain

For Retrieval Augmented Generation (RAG) in LangChain we need to initialize a `RetrievalQA` object.

For both of these we need an LLM (which we have initialized) and a LangChain vector store object.



In [ ]:
## this chunking procedure was already performed earlier
## putting it here again as a reminder

# from langchain.vectorstores import Chroma
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=200,
#                                             separators=["\n\n", "\n", " ", ""]
#                                                ) #to split docs into chunks

# docs = text_splitter.split_documents(base_docs) #the documents retrieved from arXiV earlier

# vectorstore = Chroma.from_documents(docs, embed_model) #store chunks in vector db

Confirm that the vector database has successfully stored relevant documents by testing a simple query, based on the `topic` defined earlier.

In [ ]:

query = f'what makes {topic} special?'

similar_docs = vectorstore.similarity_search(
    query,  # the search query
    k=7  # returns top  most relevant chunks of text
)

for d in similar_docs:
  print(d.page_content)
  print()

  #TODO: sometimes there will be repeated chunks: why?

that support vector machines are qualitatively robust. Since support
vector machines can be represented by a functional on the set of all

Support Vector Machines can be thought of as a method for constructing a
special kind of rule, called a linear classiﬁer, in a way that produces classiﬁers

lead to large errors in the support vector machines but also that even the
ﬁnite sample distribution of support vector machines is hardly aﬀected.

ﬁnite sample distribution of support vector machines is hardly aﬀected.
In contrast to that, we also showed that support vector machines are

Support vector machines have been discussed in this section. A support vector machine is 
a machine-learning technique, based on the guideline of structural risk minimization, which

then support vector machines are not qualitatively robust any more under
extremely mild conditions. This is demonstrated in Section 5.2 in the Ap-

technology, there are many limitations to support vector machines. There is a large

In [ ]:
#retrieved context is a list of documents

similar_docs

[Document(page_content='that support vector machines are qualitatively robust. Since support\nvector machines can be represented by a functional on the set of all', metadata={'Authors': 'Robert Hable, Andreas Christmann', 'Published': '2011-11-03', 'Summary': "Support vector machines have attracted much attention in theoretical and in\napplied statistics. Main topics of recent interest are consistency, learning\nrates and robustness. In this article, it is shown that support vector machines\nare qualitatively robust. Since support vector machines can be represented by a\nfunctional on the set of all probability measures, qualitative robustness is\nproven by showing that this functional is continuous with respect to the\ntopology generated by weak convergence of probability measures. Combined with\nthe existence and uniqueness of support vector machines, our results show that\nsupport vector machines are the solutions of a well-posed mathematical problem\nin Hadamard's sense.", 'Title':

In [ ]:
# Create QA chain

from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)



In [ ]:
# without RAG

normal_res = llm(f'what is so special about {topic}?')

print(normal_res)



Support Vector Machines (SVMs) are a type of supervised learning algorithm that can be used for classification and regression tasks. SVMs aim to find the optimal hyperplane that separates the data into different classes or predicts the output value for new inputs. Here are some key features that make SVMs special:

1. Maximum Margin Classifier: SVMs aim to find the hyperplane that maximizes the margin between the classes. The margin is the distance between the hyperplane and the nearest data points on both sides of the hyperplane. By maximizing the margin, SVMs can achieve better generalization performance.
2. Robust to Noise: SVMs are robust to noisy data because they use a kernel function to transform the data into a higher-dimensional space where noise is less relevant. This makes SVMs more resistant to outliers and noisy data compared to other algorithms like linear regression.
3. Flexible Kernels: SVMs offer a variety of kernel functions, such as linear, polynomial, radial basis

In [ ]:
# with RAG, quality of answer dependent on the retrieved context

# quality of answer also increases when number of docs retrieved increases,
# and when the max_tokens allowed to be generated in output increases

rag_res = rag_pipeline(f'what is so special about {topic}?') #returns a dictionary

print(rag_res['result'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Support Vector Machines are qualitatively robust, meaning they are less sensitive to outliers and noisy data compared to other machine learning techniques. This is because SVMs can be represented as a functional on the set of all possible rules, allowing them to produce classifiers that are more robust to deviations from the expected pattern. Additionally, the finite sample distribution of SVMs is only slightly affected by large errors, making them more reliable in situations where data is limited or noisy.


In [ ]:
rag_res

{'query': 'what is so special about Support Vector Machines?',
 'result': ' Support Vector Machines are qualitatively robust, meaning they are less sensitive to outliers and noisy data compared to other machine learning techniques. This is because SVMs can be represented as a functional on the set of all possible rules, allowing them to produce classifiers that are more robust to deviations from the expected pattern. Additionally, the finite sample distribution of SVMs is only slightly affected by large errors, making them more reliable in situations where data is limited or noisy.',
 'source_documents': [Document(page_content='that support vector machines are qualitatively robust. Since support\nvector machines can be represented by a functional on the set of all', metadata={'Authors': 'Robert Hable, Andreas Christmann', 'Published': '2011-11-03', 'Summary': "Support vector machines have attracted much attention in theoretical and in\napplied statistics. Main topics of recent interest

Feed the RAG pipeline with multiple questions

In [ ]:
# test on multiple questions about SVMs, manually generated

questions = [
    "Provide an example of a situation where applying the kernel trick is advantageous, and discuss the computational benefits.",

    "Discuss the challenges posed by imbalanced datasets in the context of support vector machines.",

    "Explain strategies to mitigate the impact of class imbalance when training support vector machines.",

    "What are the differences between support vector machines and other popular classification algorithms such as logistic regression or decision trees?",

    "In what scenarios are support vector machines likely to outperform or underperform compared to alternative methods?"
]

In [ ]:
queries = []
answers = []
contexts = []

for q in questions:
  output = rag_pipeline(q)
  queries.append(output['query'])
  answers.append(output['result'])
  source_docs = output['source_documents']

  context_list = []
  for doc in source_docs:
    # context_list.append(" ".join(doc.metadata['Summary'].split('\n'))) #only the summary included. Other parts of the page content include information such as author, publication year, which may affect evaluation
    context_list.append(doc.metadata['Summary'])                                                                  # remove the newline chars used to format output

  contexts.append(context_list)

In [ ]:
for i in range(len(answers)):
  print(queries[i].strip())
  print(answers[i].strip())
  print(contexts[i])
  print()

Provide an example of a situation where applying the kernel trick is advantageous, and discuss the computational benefits.
The kernel trick can be applied when dealing with high-dimensional data or complex models. For instance, in image recognition tasks, the feature space can be very large, making it difficult to compute pairwise similarities between images. By using the kernel trick, we can map the data into a lower-dimensional space, reducing the computational complexity of the similarity computations. Additionally, the kernel trick allows us to use non-linear models, such as support vector machines (SVMs), which can capture more subtle patterns in the data, leading to improved performance on complex tasks.
["Support vector machines have attracted much attention in theoretical and in\napplied statistics. Main topics of recent interest are consistency, learning\nrates and robustness. In this article, it is shown that support vector machines\nare qualitatively robust. Since support ve

In [ ]:
with open('samples.txt', 'w') as o:
  for i in range(len(answers)):
    o.write('\n'.join([queries[i], answers[i]]))
    o.write('\n')

_____

## Evaluate the output using RAGAS

By default, the metrics are evaluated using OpenAI API. Since we would like to use Llama2 models for evaluation, some extra set up is required

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [ ]:
context_recall.llm = llm
context_recall

ContextRecall(batch_size=15, llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7ec09ac746a0>), name='context_recall', evaluation_mode=<EvaluationMode.qcg: 7>)

In [ ]:

answer_relevancy.llm = llm
answer_relevancy.embeddings = embed_model #uses OpenAI embeddings by default
answer_relevancy


AnswerRelevancy(batch_size=15, llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7ec09ac746a0>), name='answer_relevancy', evaluation_mode=<EvaluationMode.qac: 1>, strictness=3, embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cuda:0'}, encode_kwargs={'device': 'cuda:0', 'batch_size': 16}, multi_process=False))

In [ ]:

faithfulness.llm = llm
faithfulness


Faithfulness(batch_size=15, llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7ec09ac746a0>), name='faithfulness', evaluation_mode=<EvaluationMode.qac: 1>)

In [ ]:
from datasets import Dataset

#create a huggingface data set
ds = Dataset.from_dict({"question": queries, 'contexts': contexts, "answer": answers})


In [ ]:
type(ds[1]['contexts'])

list

In [ ]:

# evaluate
from ragas.evaluation import evaluate

result = evaluate(
    ds,
    metrics=[faithfulness],
)

result

evaluating with [faithfulness]


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  0%|          | 0/1 [00:00<?, ?it/s]


TypeError: can only concatenate str (not "ChatPromptTemplate") to str